In [ ]:
# %load trafficlib.py
import sys
import numpy as np

def initroad(road, density, seedval):

    # Here we expect a road without halos

    n = len(road)

    np.random.seed(seedval)

    rng = np.random.random(n)

    road[0:n] = np.where(rng[:] < density, 1, 0)

    ncar = np.sum(road)
    
    return ncar


def updateroad(newroad, oldroad):

    n = len(oldroad)-2

    newroad[1:n+1] = np.where(oldroad[1:n+1]==0, oldroad[0:n], oldroad[2:n+2])

    nmove = (newroad[1:n+1] != oldroad[1:n+1]).sum(dtype=int)
    nmove = nmove / 2

    return nmove


def updatebcs(road):

    n = len(road)-2

    road[0]   = road[n]
    road[n+1] = road[1]


import time

def gettime():

    return time.time()


In [4]:
# %load traffic.py
#!/usr/bin/env python

import sys
import time
import numpy as np

from mpi4py import MPI

from trafficlib import initroad, updatebcs, updateroad, gettime

def main(argv):

    comm = MPI.COMM_WORLD

    size = comm.Get_size()
    rank = comm.Get_rank()

# Simulation parameters
    seedval = 5743
    ncell = 10240000
    maxiter = 1024000000//ncell
    printfreq = maxiter//10

    nlocal = ncell//size

    bigroad  = np.zeros(ncell,dtype=np.int32)
    newroad  = np.zeros(nlocal+2,dtype=np.int32)
    oldroad  = np.zeros(nlocal+2,dtype=np.int32)

    sbuf = np.zeros(1)
    rbuf = np.zeros(1)

    density = 0.52

    if (rank == 0):

        print(f"Length of road is {ncell}")
        print(f"Number of iterations is {maxiter}")
        print(f"Target density of cars is {density}")
        print(f"Running on {size} process(es)")

        # Initialise road accordingly using random number generator
        print(f"Initialising ...")

        ncars = initroad(bigroad, density, seedval)

        print(f"Actual Density of cars is {format(float(ncars)/float(ncell))}\n")
        print(f"Scattering data ...")

    comm.Scatter(bigroad, oldroad[1:nlocal+1], root=0)

    if (rank == 0):
        print(f"... done\n")

    # Compute neighbours

    rankup   = (rank + 1)
    rankdown = (rank - 1)

    # Wrap-around for cyclic boundary conditions, i.e. a roundabout

    if (rankup == size):
        rankup = 0

    if (rankdown == -1):
        rankdown = size-1

    nmove = 0
    nmovelocal = 0

    comm.barrier()
        
    tstart = MPI.Wtime()

    for iter in range(1, maxiter+1):

        comm.Sendrecv(oldroad[nlocal:nlocal+1], dest=rankup,
                      recvbuf=oldroad[0:1], source=rankdown)

        comm.Sendrecv(oldroad[1:2], dest=rankdown,
                      recvbuf=oldroad[nlocal+1:nlocal+2], source=rankup)

        nmovelocal = updateroad(newroad, oldroad)

        sbuf[0] = nmovelocal
        comm.Allreduce(sbuf, rbuf)
        nmove = rbuf[0]

        # Copy new to old array
        oldroad[1:nlocal+1] = newroad[1:nlocal+1]

        if iter % printfreq == 0:

            if (rank == 0):

                print(f"At iteration {iter} average velocity is {float(nmove)/float(ncars):.6f}")

    tstop = MPI.Wtime()

    if (rank == 0):

        print(f"\nFinished\n")
        print(f"Time taken was {tstop-tstart} seconds")
        print(f"Update rate was {1.0e-6*ncell*maxiter/(tstop-tstart)} MCOPs\n")

if __name__ == "__main__":
    main(sys.argv[1:])


Length of road is 10240000
Number of iterations is 100
Target density of cars is 0.52
Running on 1 process(es)
Initialising ...
Actual Density of cars is 0.520013671875

Scattering data ...
... done

At iteration 10 average velocity is 0.789451
At iteration 20 average velocity is 0.837291
At iteration 30 average velocity is 0.858276
At iteration 40 average velocity is 0.870672
At iteration 50 average velocity is 0.879025
At iteration 60 average velocity is 0.885042
At iteration 70 average velocity is 0.889747
At iteration 80 average velocity is 0.893434
At iteration 90 average velocity is 0.896450
At iteration 100 average velocity is 0.898941

Finished

Time taken was 7.346651200001361 seconds
Update rate was 139.38323354725352 MCOPs



In [5]:
!mpiexec /timeout 20 -n 10 python traffic.py

Length of road is 10240000
Number of iterations is 100
Target density of cars is 0.52
Running on 10 process(es)
Initialising ...
Actual Density of cars is 0.520013671875

Scattering data ...
... done

At iteration 10 average velocity is 0.789451
At iteration 20 average velocity is 0.837291
At iteration 30 average velocity is 0.858276
At iteration 40 average velocity is 0.870672
At iteration 50 average velocity is 0.879025
At iteration 60 average velocity is 0.885042
At iteration 70 average velocity is 0.889747
At iteration 80 average velocity is 0.893434
At iteration 90 average velocity is 0.896450
At iteration 100 average velocity is 0.898941

Finished

Time taken was 4.714214499981608 seconds
Update rate was 217.2154024820031 MCOPs



In [ ]:
!mpiexec

In [ ]:
!mpiexec /timeout 10 -n 1 python traffic.py